# NYC Measles Cases by Neighborhood (2018 - 2019)

* This Jupyter/Python notebook creates geo/map visualizations of the 2018-2019 NYC Measles Cases by Neighborhood
* This notebook is part of the [Visualizing the 2019 Measles Outbreak](https://carlos-afonso.github.io/measles/) GitHub project.
* [Carlos Afonso](https://www.linkedin.com/in/carlos-afonso-w/), August 26, 2019.

## Import dependencies

In [1]:
# Import the necessary libraries/modules
from datetime import datetime
import folium
import imageio
import os
import pandas as pd
import re
import xlrd

## Read all data

This section reads all the necessary data, namelly the measles by neighborhood data and the neighborhood coordinates data.

The measles by neighborhood data has been manually collected/updated from the source ([NYC Health Measles webpage](https://www1.nyc.gov/site/doh/health/health-topics/measles.page)) and saved in a spreadsheet (.xlsx) file. Each data version/update is saved in a separate sheet, and the sheets are named with the date of the update using the ISO format YYYY-MM-DD. Note that we use this manual approach because the data is small and updated infrequently (usually once a week).

The neighborhood coordinates data was collected with simple google queries (e.g., "Williamsburg, NY coordinates") and saved in a CSV file.

In [2]:
# Define data files paths
measles_data_file = os.path.join('..', 'data', 'nyc-measles-cases-by-neighborhood.xlsx')
coordinates_data_file = os.path.join('..', 'data', 'nyc-neighborhoods-coordinates.csv')

# Read measles data
if os.path.exists(measles_data_file):
    
    # Get just the sheet names (without loading all the data)
    sheet_names = xlrd.open_workbook(measles_data_file, on_demand=True).sheet_names()
    print('Info: Measles data versions available:', sheet_names)
    
    # Note: Each sheet is a data version/update and is named with the date of that update,
    # using the ISO format YYYY-MM-DD.

    # Find the sheet with the latest data
    latest_sheet = sorted(sheet_names)[-1]
    
    # Read the latest data
    measles_df = pd.read_excel(measles_data_file, sheet_name=latest_sheet)
    print('Done: Read the lastest measles data:', latest_sheet)
    
else:
    print('ERROR: measles data file not found:', measles_data_file)

# Read coordinates data
if os.path.exists(coordinates_data_file):    
    coordinates_df = pd.read_csv(coordinates_data_file)
    print('Done: Read the coordinates data')
else:
    print('ERROR: coordinates data file not found:', measles_data_file)

Info: Measles data versions available: ['2019-08-19', '2019-07-29', '2019-07-22']
Done: Read the lastest measles data: 2019-08-19
Done: Read the coordinates data


## Check measles data

The measles data is shown below. These are the details about the four data columns:

* `neighborhood`: neighborhood name
* `ongoing transmission (update_date)`:
    * **1** means "neighborhood **with** ongoing measles transmission", as of the `update_date`;
    * **0** means "neighborhood **without** ongoing measles transmission", as of the `update_date`.
* `all cases (all_start_date to update_date)`: number of **all** measles cases in that neighborhood during the current outbreak (from `all_start_date` to `update_date`).
* `latest cases (latest_start_date to update_date)`: number of the **latest** measles cases in that neighborhood (from `latest_start_date` to `update_date`).

Note that some of the data entries/rows are for a set of neighborhoods (e.g., "Brighton Beach/Coney Island"), because this is how NYC Health reports the data.

Also, note that the last row is a TOTAL count.

In [3]:
# Show all the measles data
measles_df

,neighborhood,ongoing transmission (2019-08-19),all cases (2018-09-01 to 2019-08-19),latest cases (2019-08-12 to 2019-08-19)
0,Borough Park,1,123,0
1,Crown Heights,1,8,0
2,Williamsburg,1,472,1
3,Bensonhurst,0,3,0
4,Brighton Beach/Coney Island,0,5,0
5,Chelsea/Clinton,0,1,0
6,Far Rockaway,0,1,0
7,Flatbush,0,1,0
8,Flushing,0,3,0
9,Hunts Point/Longwood/Melrose,0,1,0


## Check coordinates data

The neighborhood coordinates data is shown below, and it contains three simple columns: neighborhood name, latitute and longitude.

Note that some of the data entries/rows are for a set of neighborhoods (e.g., "Brighton Beach/Coney Island"). This is because NYC Health reports the measles data using those neighborhood sets. The latitude/longitude of a set of neighborhoods is the average of the individual neighborhoods latitudes/longitudes.

In [4]:
# Show all the coordinates data
coordinates_df

,neighborhood,latitude,longitude
0,Bensonhurst,40.6139,-73.9922
1,Borough Park,40.6350,-73.9921
2,Brighton Beach,40.5781,-73.9597
3,Coney Island,40.5755,-73.9707
4,Brighton Beach/Coney Island,40.5768,-73.9652
5,Chelsea,40.7465,-74.0014
6,Clinton,40.7638,-73.9918
7,Chelsea/Clinton,40.7552,-73.9966
8,Crown Heights,40.6694,-73.9422
9,Far Rockaway,40.5999,-73.7448


## Extract reference dates

Extract reference dates from the original column names, and transform them into nicelly formatted strings.

Extract reference dates from the original column names, and transform them into nicelly formatted strings.

In [5]:
measles_original_cols = measles_df.columns

measles_original_cols

Index(['neighborhood', 'ongoing transmission (2019-08-19)',
       'all cases (2018-09-01 to 2019-08-19)',
       'latest cases (2019-08-12 to 2019-08-19)'],
      dtype='object')

In [6]:
# Notes about the lambda function below:
# - 1. the search 
# TODO
[update_date_label, all_start_date_label, latest_start_date_label] = list(map(
    lambda x: datetime.strptime(re.search(r'\d{4}-\d{2}-\d{2}', x).group(), '%Y-%m-%d')\
        .strftime('%b %d, %Y')\
        .replace(' 0', ' '), 
    measles_original_cols[1:4]
))

[update_date_label, all_start_date_label, latest_start_date_label]

['Aug 19, 2019', 'Sep 1, 2018', 'Aug 12, 2019']

## Rename measles data columns

Three of the columns names contain useful date information which we will want to shown in the visualizations, to provide context. However, that makes those column names very long/verbose. So, let's first save those original column names, and then rename them with shorter names, to make it easier to reference them.

In [7]:
measles_new_cols = ['neighborhood', 'ongoing_transmission', 'all_cases', 'latest_cases']

measles_df.columns = measles_new_cols

measles_df.head(1)

,neighborhood,ongoing_transmission,all_cases,latest_cases
0,Borough Park,1,123,0


## Extract totals

In [8]:
all_cases_total = measles_df[measles_df['neighborhood'] == 'TOTAL']['all_cases'].values[0]

latest_cases_total = measles_df[measles_df['neighborhood'] == 'TOTAL']['latest_cases'].values[0]

[all_cases_total, latest_cases_total]

[654, 1]

## Merge measles and coordinates data

In [9]:
# Merge the measles and coordinates data into a single dataframe
df = pd.merge(measles_df, coordinates_df, on='neighborhood')

# Show the data
df

,neighborhood,ongoing_transmission,all_cases,latest_cases,latitude,longitude
0,Borough Park,1,123,0,40.6350,-73.9921
1,Crown Heights,1,8,0,40.6694,-73.9422
2,Williamsburg,1,472,1,40.7081,-73.9571
3,Bensonhurst,0,3,0,40.6139,-73.9922
4,Brighton Beach/Coney Island,0,5,0,40.5768,-73.9652
5,Chelsea/Clinton,0,1,0,40.7552,-73.9966
6,Far Rockaway,0,1,0,40.5999,-73.7448
7,Flatbush,0,1,0,40.6415,-73.9594
8,Flushing,0,3,0,40.7675,-73.8331
9,Hunts Point/Longwood/Melrose,0,1,0,40.8196,-73.8941


## Add columns with color, radius, and label coordinates 

In [10]:
# Add color column

active_color   = 'red'  # '#ff4d4d' '#ff0000' '#8b0000'
inactive_color = 'blue' # '#4d4dff' '#0000ff' '#00008b'

df['ongoing_transmission_color'] = list(map(
    lambda x: active_color if x == 1 else inactive_color, 
    df['ongoing_transmission']
))

# Add radius columns

def cases_radius(x):
    if x == 0:
        return 2
    elif x < 10:
        return 8
    elif x < 100:
        return 16
    else:
        return 24

df['all_cases_radius'] = list(map(cases_radius, df['all_cases']))

df['latest_cases_radius'] = list(map(cases_radius, df['latest_cases']))

# Add label coordinates columns

df['label_latitude'] = df['latitude']

df['label_longitude'] = df['longitude']

df

,neighborhood,ongoing_transmission,all_cases,latest_cases,latitude,longitude,ongoing_transmission_color,all_cases_radius,latest_cases_radius,label_latitude,label_longitude
0,Borough Park,1,123,0,40.6350,-73.9921,red,24,2,40.6350,-73.9921
1,Crown Heights,1,8,0,40.6694,-73.9422,red,8,2,40.6694,-73.9422
2,Williamsburg,1,472,1,40.7081,-73.9571,red,24,8,40.7081,-73.9571
3,Bensonhurst,0,3,0,40.6139,-73.9922,blue,8,2,40.6139,-73.9922
4,Brighton Beach/Coney Island,0,5,0,40.5768,-73.9652,blue,8,2,40.5768,-73.9652
5,Chelsea/Clinton,0,1,0,40.7552,-73.9966,blue,8,2,40.7552,-73.9966
6,Far Rockaway,0,1,0,40.5999,-73.7448,blue,8,2,40.5999,-73.7448
7,Flatbush,0,1,0,40.6415,-73.9594,blue,8,2,40.6415,-73.9594
8,Flushing,0,3,0,40.7675,-73.8331,blue,8,2,40.7675,-73.8331
9,Hunts Point/Longwood/Melrose,0,1,0,40.8196,-73.8941,blue,8,2,40.8196,-73.8941


In [11]:
df.shape

(18, 11)

## Adjust the label coordinates manually

In [12]:
df['neighborhood']

0                     Borough Park
1                    Crown Heights
2                     Williamsburg
3                      Bensonhurst
4      Brighton Beach/Coney Island
5                  Chelsea/Clinton
6                     Far Rockaway
7                         Flatbush
8                         Flushing
9     Hunts Point/Longwood/Melrose
10                         Jamaica
11        Long Island City/Astoria
12             Midwood/Marine Park
13                   Port Richmond
14                        Red Hook
15                     Sunset Park
16                     West Queens
17                     Willowbrook
Name: neighborhood, dtype: object

In [13]:
df.loc[df['neighborhood'] == 'Bensonhurst',                  ['label_latitude', 'label_longitude']] = [40.6,   -74.03]
df.loc[df['neighborhood'] == 'Borough Park',                 ['label_latitude', 'label_longitude']] = [40.63,  -74.095]
df.loc[df['neighborhood'] == 'Brighton Beach/Coney Island',  ['label_latitude', 'label_longitude']] = [40.565, -74.03]
df.loc[df['neighborhood'] == 'Chelsea/Clinton',              ['label_latitude', 'label_longitude']] = [40.745, -74.035]
df.loc[df['neighborhood'] == 'Crown Heights',                ['label_latitude', 'label_longitude']] = [40.669, -73.927]
df.loc[df['neighborhood'] == 'Far Rockaway',                 ['label_latitude', 'label_longitude']] = [40.585, -73.81]
df.loc[df['neighborhood'] == 'Flatbush',                     ['label_latitude', 'label_longitude']] = [40.641, -73.945]
df.loc[df['neighborhood'] == 'Flushing',                     ['label_latitude', 'label_longitude']] = [40.777, -73.85]
df.loc[df['neighborhood'] == 'Hunts Point/Longwood/Melrose', ['label_latitude', 'label_longitude']] = [40.807, -73.97]
df.loc[df['neighborhood'] == 'Jamaica',                      ['label_latitude', 'label_longitude']] = [40.69,  -73.81]
df.loc[df['neighborhood'] == 'Long Island City/Astoria',     ['label_latitude', 'label_longitude']] = [40.765, -73.98]
df.loc[df['neighborhood'] == 'Midwood/Marine Park',          ['label_latitude', 'label_longitude']] = [40.615, -73.93]
df.loc[df['neighborhood'] == 'Port Richmond',                ['label_latitude', 'label_longitude']] = [40.648, -74.15]
df.loc[df['neighborhood'] == 'Red Hook',                     ['label_latitude', 'label_longitude']] = [40.685, -74.03]
df.loc[df['neighborhood'] == 'Sunset Park',                  ['label_latitude', 'label_longitude']] = [40.663, -74.09]
df.loc[df['neighborhood'] == 'West Queens',                  ['label_latitude', 'label_longitude']] = [40.733,  -73.92]
df.loc[df['neighborhood'] == 'Williamsburg',                 ['label_latitude', 'label_longitude']] = [40.708, -73.93]
df.loc[df['neighborhood'] == 'Willowbrook',                  ['label_latitude', 'label_longitude']] = [40.59,  -74.15]

## Define function to create the maps

In [18]:
def cases_map_function(data, all_or_latest):
    
    if all_or_latest == 'all':
        title_prefix     = '<b>All</b>'
        cases_total      = all_cases_total
        start_date_label = all_start_date_label
        cases_col        = 'all_cases'
        radius_col       = 'all_cases_radius'
    elif all_or_latest == 'latest':
        title_prefix     = '<b>Latest</b>'
        cases_total      = latest_cases_total
        start_date_label = latest_start_date_label
        cases_col        = 'latest_cases'
        radius_col       = 'latest_cases_radius'
    else:
        return 'ERROR: invalid all_or_latest argument (it should be "all" or "latest").'
    
    # check if the start and update date labels have the same year
    if start_date_label[-6:] == update_date_label[-6:]:
        # if True, remove the year part from start date label 
        start_date_label = start_date_label[:-6]
    else:
        # if False, add a comma to the end of the start date label
        start_date_label = start_date_label + ','
    
    # Start figure
    fig = folium.Figure()
    
    # Figure title
    fig.html.add_child(folium.Element('<h3>' + title_prefix + ' NYC Measles Cases by Neighborhood</h3>'))
    
    # Figure sub-title
    fig.html.add_child(folium.Element('<h4><b>' + str(cases_total) + ' total cases from ' + start_date_label + ' to ' + update_date_label + '</b></h4>'))
    
    # Note about data and image sources
    fig.html.add_child(folium.Element('<h5 style="font-family:consolas;">Data: NYC Health, Image: carlos-afonso.github.io/measles</h5>'))
    
    # Base map
    cases_map = folium.Map(
        location   = [40.69, -73.945], 
        # tiles: OpenStreetMap, Mapbox Bright, Mapbox Control Room, Stamen (Terrain, Toner, and Watercolor)
        #tiles      = 'Mapbox Bright', 
        tiles      = 'CartoDBPositron', 
        height     = 540, 
        width      = 620, 
        zoom_start = 11, 
        minZoom = 11, 
        maxZoom = 11, 
        dragging = False, 
        doubleClickZoom = False, 
        scrollWheelZoom = False, 
        touchZoom = False, 
        zoom_control = False
    )
    
    cond = (df[cases_col] > 0) | (df['ongoing_transmission'] == 1)
    
    for index, row in df[cond].iterrows():
        # Draw circles
        folium.CircleMarker(
            location = [row['latitude'], row['longitude']], 
            radius = row[radius_col], 
            #tooltip = row['neighborhood'] + ' (' + str(row[cases_col]) + ')', 
            color = row['ongoing_transmission_color'], 
            fill_color = row['ongoing_transmission_color']
        ).add_to(cases_map)
        # Write labels
        folium.Marker(
            location = [row['label_latitude'], row['label_longitude']], 
            icon = folium.DivIcon(html = (
                '<svg>'
                '<text x="0" y="10" fill=' + row['ongoing_transmission_color'] + ' font-size="14" font-weight="bold">' + 
                    row['neighborhood'].replace('/', ' / ') + ' (' + str(row[cases_col]) + ')</text>'
                '</svg>'
            ))
        ).add_to(cases_map)
    
    # Create "Size Legend"
    
    # Legend box (rectangle)
    folium.Marker(
        location = [40.825, -74.15], 
        icon = folium.DivIcon(html = (
            '<svg  width="135" height="165">'
            '<rect width="135" height="165" fill="white" fill-opacity="0.0" stroke="black" stroke-width="2"/>'
            '</svg>'
        ))
    ).add_to(cases_map)
    
    # Legend title
    #folium.Marker(
    #    location = [40.822, -74.124], 
    #    icon = folium.DivIcon(html = (
    #        '<svg><text x="0" y="10" fill="black" font-size="14" font-weight="bold">Size Legend</text></svg>'
    #    ))
    #).add_to(cases_map)
    
    # Create circles
    folium.Marker(
        location = [40.82, -74.14], 
        icon = folium.DivIcon(html = (
            '<svg><text x="0" y="10" fill="black" font-size="12" font-weight="bold">Size</text></svg>'
        ))
    ).add_to(cases_map)
    folium.CircleMarker(
        location = [40.81, -74.134], radius = 2, color = 'black', fill_color = 'black'
    ).add_to(cases_map)
    folium.CircleMarker(
        location = [40.80, -74.134], radius = 8, color = 'black', fill_color = 'black'
    ).add_to(cases_map)
    folium.CircleMarker(
        location = [40.783, -74.134], radius = 16, color = 'black', fill_color = 'black'
    ).add_to(cases_map)
    folium.CircleMarker(
        location = [40.76, -74.134], radius = 24, color = 'black', fill_color = 'black'
    ).add_to(cases_map)
    
    # Add labels in front of circles
    folium.Marker(
        location = [40.82, -74.11], 
        icon = folium.DivIcon(html = (
            '<svg><text x="0" y="10" fill="black" font-size="12" font-weight="bold">#Cases</text></svg>'
        ))
    ).add_to(cases_map)
    folium.Marker(
        location = [40.81, -74.11], 
        icon = folium.DivIcon(html = (
            '<svg><text x="0" y="10" fill="black" font-size="12" font-weight="bold">0</text></svg>'
        ))
    ).add_to(cases_map)
    folium.Marker(
        location = [40.80, -74.11], 
        icon = folium.DivIcon(html = (
            '<svg><text x="0" y="10" fill="black" font-size="12" font-weight="bold">1 to 9</text></svg>'
        ))
    ).add_to(cases_map)
    folium.Marker(
        location = [40.783, -74.11], 
        icon = folium.DivIcon(html = (
            '<svg><text x="0" y="10" fill="black" font-size="12" font-weight="bold">10 to 99</text></svg>'
        ))
    ).add_to(cases_map)
    folium.Marker(
        location = [40.76, -74.11], 
        icon = folium.DivIcon(html = (
            '<svg><text x="0" y="10" fill="black" font-size="12" font-weight="bold">100 or more</text></svg>'
        ))
    ).add_to(cases_map)    
    
    # Create "Color Legend"
    
    # Legend box (rectangle)
    folium.Marker(
        location = [40.72, -74.15], 
        icon = folium.DivIcon(html = (
            '<svg  width="145" height="50">'
            '<rect width="145" height="50" fill="white" fill-opacity="0.0" stroke="black" stroke-width="2"/>'
            '</svg>'
        ))
    ).add_to(cases_map)
    
    # Legend title
    #folium.Marker(
    #    location = [40.715, -74.144], 
    #    icon = folium.DivIcon(html = (
    #        '<svg><text x="0" y="10" fill="black" font-size="13" font-weight="bold">Neighborhood</text></svg>'
    #    ))
    #).add_to(cases_map)
    
    # Legend labels
    folium.Marker(
        location = [40.715, -74.148], 
        icon = folium.DivIcon(html = (
            '<svg>\
                <text x="0" y="10" fill="black" font-size="12" font-weight="bold">Ongoing Transmission?</text>\
                <text x="20" y="30" fill="red" font-size="12" font-weight="bold">YES</text>\
                <text x="55" y="30" fill="black" font-size="14" font-weight="bold"> / </text>\
                <text x="70" y="30" fill="blue" font-size="12" font-weight="bold">NO</text>\
            </svg>'
        ))
    ).add_to(cases_map)
    #folium.Marker(
    #    location = [40.695, -74.145], 
    #    icon = folium.DivIcon(html = (
    #        '<svg><text x="0" y="10" fill="blue" font-size="12" font-weight="bold">Without Ongoing Transmission</text></svg>'
    #    ))
    #).add_to(cases_map)

    fig.add_child(cases_map)
    
    return fig

## Create All Cases Map

In [19]:
total_cases_map = cases_map_function(df, 'all')
total_cases_map.save('../images/nyc-measles-cases-by-neighborhood-map-all-py.html')
total_cases_map

## Create Latest Cases Map

In [20]:
new_cases_map = cases_map_function(df, 'latest')

new_cases_map.save('../images/nyc-measles-cases-by-neighborhood-map-latest-py.html')

new_cases_map

## Transform the Maps from HTML to PNG

At the moment this step is done "manually", as follows:
* In Firefox:
    * Open the HTML file/image in Firefox
    * Using the "Take a Screenshot" feature in Firefox, take a screenshot of the visible area, and download/save it as a PNG image.
* In GIMP:
    * Open the screenshot (PNG) image in GIMP
    * Auto-crop the image
    * Export/Save the result as a PNG image

## Create All & New Cases Map (GIF)

In [21]:
images = [
    imageio.imread('../images/nyc-measles-cases-by-neighborhood-map-all-py.png'), 
    imageio.imread('../images/nyc-measles-cases-by-neighborhood-map-latest-py.png'), 
]

imageio.mimsave('../images/nyc-measles-cases-by-neighborhood-map-all-latest-py.gif', images, duration = 5)